In [1]:
import os
import random
import matplotlib.pyplot as plt

In [2]:
os.chdir("../")
%pwd

'/home/megh/AI/Projects/Traffic_signs_detection'

In [3]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_dir:Path
    train_images: Path
    test_images: Path
    train_labels: Path
    test_labels: Path
    train_test_split: float
    train_val_split : float

In [4]:
from src.constants import *

In [5]:
from src.utils.common import read_yaml,create_directories

In [6]:
from random import choice
import shutil

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
            config = self.config.data_ingestions
            params = self.params
            
            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                root_dir= config.root_dir,
                    source_dir= config.source_dir,
                    train_images= config.train_images,
                    test_images= config.test_images,
                    val_images= config.val_images,
                    train_labels= config.train_labels,
                    test_labels= config.test_labels,
                    val_labels= config.val_labels,
                    train_test_split = params.TRAIN_TEST_SPLIT,
                    train_val_split= params.TRAIN_VAL_SPLIT
            )

            return data_ingestion_config


In [8]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config=config


    def prepare_data(self):

        imgs = []
        labels = []

        for (dirname, dirs, files) in os.walk(self.config.source_dir):
            for filename in files:
                if filename.endswith(".txt"):
                    labels.append(filename)
                else:
                    imgs.append(filename)


        #Creating images and label directories for Train, val and test
        if not os.path.isdir(self.config.train_images):
            create_directories([self.config.train_images])

        if not os.path.isdir(self.config.train_labels):
            create_directories([self.config.train_labels])

        if not os.path.isdir(self.config.test_images):
            create_directories([self.config.test_images])

        if not os.path.isdir(self.config.test_labels):
            create_directories([self.config.test_labels])


        if not os.path.isdir(self.config.val_images):
            create_directories([self.config.val_images])

        if not os.path.isdir(self.config.val_labels):
            create_directories([self.config.val_labels])

        test_count = self.config.train_test_split * len(imgs)
        val_count = self.config.train_val_split * len(imgs)
        train_count = len(imgs) - test_count - val_count
        print(test_count, val_count, train_count)


        for i in range(train_count):
            file_jpg = choice(imgs)
            file_txt= file_jpg[:-4]+".txt"

            shutil.copy(os.path.join(self.config.source_dir,file_jpg), os.path.join(self.config.train_images,file_jpg))
            shutil.copy(os.path.join(self.config.source_dir,file_txt), os.path.join(self.config.train_labels,file_txt))

            imgs.remove(file_jpg)
            labels.remove(file_txt)


        for i in range(val_count):
            file_jpg = choice(imgs)
            file_txt= file_jpg[:-4]+".txt"

            shutil.copy(os.path.join(self.config.source_dir,file_jpg), os.path.join(self.config.val_images,file_jpg))
            shutil.copy(os.path.join(self.config.source_dir,file_txt), os.path.join(self.config.val_labels,file_txt))

            imgs.remove(file_jpg)
            labels.remove(file_txt)


        for i in range(test_count):
            file_jpg = choice(imgs)
            file_txt= file_jpg[:-4]+".txt"

            shutil.copy(os.path.join(self.config.source_dir,file_jpg), os.path.join(self.config.test_images,file_jpg))
            shutil.copy(os.path.join(self.config.source_dir,file_txt), os.path.join(self.config.test_labels,file_txt))

            imgs.remove(file_jpg)
            labels.remove(file_txt)

        


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config = data_ingestion_config)
    data_ingestion.prepare_data()

except Exception as e:
    raise e

[2023-09-12 21:07:41,642: INFO: common: yaml fileconfig.yamlloaded Successfully]
[2023-09-12 21:07:41,644: INFO: common: yaml fileparams.yamlloaded Successfully]
[2023-09-12 21:07:41,644: INFO: common: created directory at:artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'data_ingestion'"